In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=b519764e9d99efa62d3f7d64f1367843521840f2e90580ed8036014df47e5cef
  Stored in directory: /home/spark/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [7]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 17:40:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [9]:
sc = spark.sparkContext

In [10]:
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

tadeusz_lines = pan_tadeusz_file.filter(lambda line: "Tadeusz" in line)

tadeusz_count = tadeusz_lines.count()

print(f"Liczba linii zawierających słowo 'Tadeusz': {tadeusz_count}")

Liczba linii zawierających słowo 'Tadeusz': 182


In [11]:
longest_lines = pan_tadeusz_file.top(3, key=lambda line: len(line))

print("3 najdłuższe linie:")
for i, line in enumerate(longest_lines, 1):
    print(f"{i}. {line}")

3 najdłuższe linie:
1. O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.
2. Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem — Kłótnia — Hrabia z Gerwazym odbywaj

In [13]:
import re
import json

stopwords = set(sc.textFile("polish.stopwords.txt").collect())

words_rdd = (
    pan_tadeusz_file
    .flatMap(lambda line: re.findall(r'\b\w+\b', line.lower()))
    .filter(lambda word: word not in stopwords)                  
)

word_counts = words_rdd.countByValue()

most_common_word = max(word_counts.items(), key=lambda x: x[1])

with open("pan_tadeusz_bag_of_words.json", "w", encoding="utf-8") as f:
    json.dump(word_counts, f, ensure_ascii=False, indent=4)

print(f"Najczęstsze słowo: {most_common_word[0]} (wystąpienia: {most_common_word[1]})")


Najczęstsze słowo: rzekł (wystąpienia: 155)
